In [ ]:
import numpy as np
from pyscf import gto, scf, ao2mo, ci, cc, fci
from pyscf.tools import fcidump

# =============================================================================
# 1. Read FCIDUMP file and setup molecular system
# =============================================================================
print("=" * 60)
print("Reading FCIDUMP file and setting up system")
print("=" * 60)

fcidump_file = '../FCIDUMP/N2_sto3g_2.50.FCIDUMP'
data = fcidump.read(fcidump_file)

# Extract integrals and system parameters
h1e = data['H1']          # One-electron integrals
h2e = data['H2']          # Two-electron integrals
n_elec = data['NELEC']    # Total number of electrons
n_orbs = data['NORB']     # Number of orbitals
e_nuc = data['ECORE']     # Nuclear repulsion energy

print(f"Number of orbitals: {n_orbs}")
print(f"Number of electrons: {n_elec}")
print(f"Nuclear repulsion energy: {e_nuc:.8f} Ha\n")

# =============================================================================
# 2. Setup molecule and RHF calculation using FCIDUMP integrals
# =============================================================================
print("=" * 60)
print("Setting up RHF calculation")
print("=" * 60)

# Create a minimal molecule object
mol = gto.Mole()
mol.nelectron = n_elec
mol.spin = 0  # Singlet state (Ms = 0)
mol.build()

# Initialize RHF object and override with FCIDUMP integrals
mf = scf.RHF(mol)
mf.energy_nuc = lambda *args: e_nuc
mf.get_hcore = lambda *args: h1e
mf.get_ovlp = lambda *args: np.identity(n_orbs)  # MO basis has identity overlap
mf._eri = ao2mo.restore(1, h2e, n_orbs)  # Restore 8-fold symmetry

# Run HF calculation (should converge in one iteration)
mf.kernel()
print(f"Hartree-Fock Energy: {mf.e_tot:.8f} Ha\n")

# =============================================================================
# 3. FCI calculation without spin constraint (problematic)
# =============================================================================
print("=" * 60)
print("FCI Calculation #1: Without spin constraint (Ms conserved only)")
print("=" * 60)

neleca = n_elec // 2
nelecb = n_elec - neleca
nelec_tuple = (neleca, nelecb)

# Use direct_spin1 solver (conserves Ms but not S)
myfci_unconstrained = fci.direct_spin1.FCI()
myfci_unconstrained.nroots = 5

# Calculate multiple roots
fci_energies_uncon, fci_vecs_uncon = myfci_unconstrained.kernel(
    h1e, h2e, n_orbs, nelec_tuple
)

print("Found roots (may contain mixed spin states):")
for i, vec in enumerate(fci_vecs_uncon):
    s2 = myfci_unconstrained.spin_square(vec, n_orbs, nelec_tuple)[0]
    total_energy = fci_energies_uncon[i] + e_nuc
    print(f"  Root {i}: E_total = {total_energy:.8f} Ha, <S²> = {s2:.4f}")
print("Note: S=0 expects <S²>=0.0, S=1 expects <S²>=2.0\n")

# =============================================================================
# 4. FCI calculation with singlet constraint (correct)
# =============================================================================
print("=" * 60)
print("FCI Calculation #2: With singlet (S=0) constraint")
print("=" * 60)

# Use fix_spin to enforce pure singlet states (ss=0 means S²=0)
myfci_singlet = fci.addons.fix_spin(fci.FCI(mol), ss=0)
myfci_singlet.nroots = 5

# Calculate singlet-only roots
fci_energies_singlet, fci_vecs_singlet = myfci_singlet.kernel(
    h1e, h2e, n_orbs, nelec_tuple
)

print("Found roots (enforced singlets):")
for i, vec in enumerate(fci_vecs_singlet):
    s2 = fci.spin_op.spin_square(vec, n_orbs, nelec_tuple)[0]
    total_energy = fci_energies_singlet[i] + e_nuc
    print(f"  Root {i}: E_total = {total_energy:.8f} Ha, <S²> = {s2:.4f}")

# =============================================================================
# 6. Summary and comparison
# =============================================================================
print("\n" + "=" * 60)
print("SUMMARY: Energy Comparison")
print("=" * 60)

print(f"Hartree-Fock:                    {mf.e_tot:.8f} Ha")
print(f"FCI (unconstrained, lowest):     {fci_energies_uncon[0] + e_nuc:.8f} Ha")
print(f"FCI (singlet, ground state):     {fci_energies_singlet[0] + e_nuc:.8f} Ha")

print("\n" + "=" * 60)
print("CONCLUSION")
print("=" * 60)
print("The singlet-constrained FCI gives the correct ground state energy")
print("for exact diagonalization in the singlet subspace.")
print("=" * 60)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pyscf import gto, scf, fci
import csv

# =============================================================================
# Configuration
# =============================================================================
# Bond lengths to scan (in Angstrom)
bond_lengths = np.concatenate([
    np.linspace(0.9, 1.1, 5),
    np.linspace(1.1, 1.5, 9),
    np.linspace(1.5, 2.5, 11),
    np.linspace(2.5, 4.0, 7)
])
bond_lengths = np.unique(np.round(bond_lengths, 4))

reference_distance = 1.2425  # Angstrom
basis = 'sto-3g'
Ha_to_eV = 27.211396  # Hartree to eV conversion

print("=" * 70)
print("C2 Dissociation Potential Energy Surface")
print("=" * 70)
print(f"Basis set: {basis}")
print(f"Number of geometries: {len(bond_lengths)}")
print(f"Reference distance: {reference_distance} Å (singlet state)")
print(f"Number of roots per state: 5 (to ensure correct ground state)")
print("=" * 70)

# =============================================================================
# Function to calculate FCI ground state energy for given spin state
# =============================================================================
def calculate_fci_ground_state(distance, spin_state):
    """
    Calculate FCI ground state energy for C2 at given distance and spin state.
    Uses nroots=5 to ensure the true ground state is found.
    
    Args:
        distance: C-C bond length in Angstrom
        spin_state: 0 (singlet) or 2 (triplet)
    
    Returns:
        tuple: (ground_state_energy, <S²>, nuclear_repulsion_energy)
    """
    # Build molecule at specified geometry
    mol = gto.Mole()
    mol.atom = f'C 0 0 0; C 0 0 {distance}'
    mol.basis = basis
    mol.spin = spin_state  # 2S value: 0 for singlet, 2 for triplet
    mol.build()
    
    n_elec = mol.nelectron
    n_orbs = mol.nao
    e_nuc = mol.energy_nuc()
    
    # Run mean-field calculation for initial guess
    if spin_state == 0:
        mf = scf.RHF(mol)
    else:
        mf = scf.ROHF(mol)
    mf.kernel()
    
    # Transform integrals to MO basis
    h1e = mf.mo_coeff.T @ mf.get_hcore() @ mf.mo_coeff
    eri_ao = mol.intor('int2e')
    h2e = np.einsum('pi,qj,pqrs,rk,sl->ijkl', 
                    mf.mo_coeff, mf.mo_coeff, eri_ao, 
                    mf.mo_coeff, mf.mo_coeff, optimize=True)
    
    # Setup electron configuration
    neleca = (n_elec + spin_state) // 2
    nelecb = (n_elec - spin_state) // 2
    nelec_tuple = (neleca, nelecb)
    
    # Configure FCI solver with spin constraint
    if spin_state == 0:
        # Enforce singlet: S² = 0
        myfci = fci.addons.fix_spin(fci.FCI(mol), ss=0)
    elif spin_state == 2:
        # Enforce triplet: S² = 2 (S=1)
        myfci = fci.addons.fix_spin(fci.FCI(mol), ss=2)
    else:
        raise ValueError("spin_state must be 0 (singlet) or 2 (triplet)")
    
    # Calculate 10 lowest roots to ensure we have the true ground state
    myfci.nroots = 10
    e_fci_list, vec_fci_list = myfci.kernel(h1e, h2e, n_orbs, nelec_tuple)
    
    # Select ground state (lowest energy)
    e_ground = e_fci_list[0]
    vec_ground = vec_fci_list[0]
    
    # Verify spin expectation value
    s2 = fci.spin_op.spin_square(vec_ground, n_orbs, nelec_tuple)[0]
    
    # Total energy includes nuclear repulsion
    e_total = e_ground + e_nuc
    
    return e_total, s2, e_nuc

# =============================================================================
# Calculate reference energy (singlet at 1.2425 Å)
# =============================================================================
print(f"\nCalculating reference energy (singlet at {reference_distance} Å)...")
ref_energy, ref_s2, ref_nuc = calculate_fci_ground_state(reference_distance, spin_state=0)
print(f"Reference energy: {ref_energy:.8f} Ha")
print(f"Reference <S²>: {ref_s2:.4f} (expected: 0.0)")
print(f"Reference E_nuc: {ref_nuc:.8f} Ha")

# Also calculate triplet at reference distance for annotation
ref_triplet, ref_s2_t, _ = calculate_fci_ground_state(reference_distance, spin_state=2)
print(f"Triplet at ref: {ref_triplet:.8f} Ha")
print(f"Triplet <S²>: {ref_s2_t:.4f} (expected: 2.0)")

# =============================================================================
# Scan potential energy surfaces for singlet and triplet states
# =============================================================================
energies_singlet = []
energies_triplet = []
s2_values_singlet = []
s2_values_triplet = []

print("\n" + "=" * 70)
print("Scanning potential energy surfaces...")
print("=" * 70)

for i, dist in enumerate(bond_lengths):
    print(f"\n[{i+1}/{len(bond_lengths)}] Distance: {dist:.4f} Å")
    
    # Singlet state (S=0)
    e_s, s2_s, nuc_s = calculate_fci_ground_state(dist, spin_state=0)
    energies_singlet.append(e_s)
    s2_values_singlet.append(s2_s)
    print(f"  Singlet: E = {e_s:.8f} Ha, <S²> = {s2_s:.4f}, E_nuc = {nuc_s:.6f} Ha")
    
    # Triplet state (S=1)
    e_t, s2_t, nuc_t = calculate_fci_ground_state(dist, spin_state=2)
    energies_triplet.append(e_t)
    s2_values_triplet.append(s2_t)
    print(f"  Triplet: E = {e_t:.8f} Ha, <S²> = {s2_t:.4f}, E_nuc = {nuc_t:.6f} Ha")

# Convert to numpy arrays
energies_singlet = np.array(energies_singlet)
energies_triplet = np.array(energies_triplet)
s2_values_singlet = np.array(s2_values_singlet)
s2_values_triplet = np.array(s2_values_triplet)

# Calculate relative energies in eV
rel_energies_singlet = (energies_singlet - ref_energy) * Ha_to_eV
rel_energies_triplet = (energies_triplet - ref_energy) * Ha_to_eV
ref_rel_triplet = (ref_triplet - ref_energy) * Ha_to_eV

# =============================================================================
# Save data to CSV file
# =============================================================================
csv_filename = 'C2_FCI_PES_data.csv'
print("\n" + "=" * 70)
print(f"Saving data to {csv_filename}...")
print("=" * 70)

with open(csv_filename, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    
    # Write header
    writer.writerow(['CC_Distance_Angstrom', 'Singlet_Energy_Hartree', 'Triplet_Energy_Hartree'])
    
    # Write data rows
    for i in range(len(bond_lengths)):
        writer.writerow([f'{bond_lengths[i]:.6f}', 
                        f'{energies_singlet[i]:.10f}', 
                        f'{energies_triplet[i]:.10f}'])

print(f"Data successfully saved to {csv_filename}")
print(f"Total rows: {len(bond_lengths)} (excluding header)")

# =============================================================================
# Create publication-quality plot
# =============================================================================
plt.rcParams.update({
    'font.size': 11,
    'axes.labelsize': 12,
    'axes.titlesize': 14,
    'lines.linewidth': 2.5,
    'grid.alpha': 0.3,
    'legend.fontsize': 11,
})

fig, ax = plt.subplots(figsize=(10, 7))

# Plot potential energy curves
ax.plot(bond_lengths, rel_energies_singlet, 'o-', 
        color='steelblue', markersize=6, markeredgecolor='white', 
        markeredgewidth=1.0, label='Singlet (S=0)', zorder=3)

ax.plot(bond_lengths, rel_energies_triplet, 's-', 
        color='crimson', markersize=6, markeredgecolor='white', 
        markeredgewidth=1.0, label='Triplet (S=1)', zorder=3)

# Mark reference point with vertical line
ax.axvline(reference_distance, color='gray', linestyle='--', 
           linewidth=1.5, alpha=0.5, zorder=1, label=f'Ref: {reference_distance} Å')

# Highlight reference point energies with larger markers
ax.plot(reference_distance, 0.0, 'o', 
        color='steelblue', markersize=12, markeredgecolor='black', 
        markeredgewidth=1.5, zorder=4)
ax.plot(reference_distance, ref_rel_triplet, 's', 
        color='crimson', markersize=12, markeredgecolor='black', 
        markeredgewidth=1.5, zorder=4)

# Add annotations for reference point
ax.annotate(f'Singlet: 0.00 eV\n<S²> = {ref_s2:.3f}', 
            xy=(reference_distance, 0.0), xytext=(reference_distance + 0.15, 0.0),
            fontsize=9, ha='left', va='center',
            bbox=dict(boxstyle='round,pad=0.3', facecolor='white', 
                     edgecolor='steelblue', alpha=0.8))

ax.annotate(f'Triplet: {ref_rel_triplet:.2f} eV\n<S²> = {ref_s2_t:.3f}', 
            xy=(reference_distance, ref_rel_triplet), 
            xytext=(reference_distance + 0.15, ref_rel_triplet),
            fontsize=9, ha='left', va='center',
            bbox=dict(boxstyle='round,pad=0.3', facecolor='white', 
                     edgecolor='crimson', alpha=0.8))

# Labels and formatting
ax.set_xlabel('C-C Distance (Å)', fontsize=13)
ax.set_ylabel(f'Relative Energy (eV)\n[Reference: Singlet @ {reference_distance} Å]', 
              fontsize=13)
ax.set_title(f'C₂ Dissociation Potential Energy Surface (FCI/{basis})', 
             fontsize=15, fontweight='bold')
ax.legend(loc='upper right', framealpha=0.95, edgecolor='gray')
ax.grid(True, alpha=0.3)

# Set y-axis limits with margin
y_min = min(rel_energies_singlet.min(), rel_energies_triplet.min())
y_max = max(rel_energies_singlet.max(), rel_energies_triplet.max())
margin = (y_max - y_min) * 0.1
ax.set_ylim(y_min - margin, y_max + margin)

plt.tight_layout()
plt.savefig('C2_PES_FCI_Singlet_Triplet.pdf', dpi=300, bbox_inches='tight')
plt.show()

# =============================================================================
# Summary statistics
# =============================================================================
print("\n" + "=" * 70)
print("SUMMARY")
print("=" * 70)

# Find minima for each state
idx_s_min = np.argmin(energies_singlet)
idx_t_min = np.argmin(energies_triplet)

print(f"\nSinglet ground state minimum:")
print(f"  Distance: {bond_lengths[idx_s_min]:.4f} Å")
print(f"  Energy: {energies_singlet[idx_s_min]:.8f} Ha")
print(f"  Relative: {rel_energies_singlet[idx_s_min]:.4f} eV")
print(f"  <S²>: {s2_values_singlet[idx_s_min]:.4f}")

print(f"\nTriplet ground state minimum:")
print(f"  Distance: {bond_lengths[idx_t_min]:.4f} Å")
print(f"  Energy: {energies_triplet[idx_t_min]:.8f} Ha")
print(f"  Relative: {rel_energies_triplet[idx_t_min]:.4f} eV")
print(f"  <S²>: {s2_values_triplet[idx_t_min]:.4f}")

# Singlet-triplet gap at reference distance
gap_ref = ref_rel_triplet
print(f"\nSinglet-triplet gap at {reference_distance} Å: {gap_ref:.4f} eV")

# Find crossing point (if any)
if np.any(np.diff(np.sign(rel_energies_triplet - rel_energies_singlet))):
    # Estimate crossing region
    diff = rel_energies_triplet - rel_energies_singlet
    sign_changes = np.where(np.diff(np.sign(diff)))[0]
    if len(sign_changes) > 0:
        cross_idx = sign_changes[0]
        cross_dist_approx = (bond_lengths[cross_idx] + bond_lengths[cross_idx + 1]) / 2
        print(f"\nApproximate singlet-triplet crossing: ~{cross_dist_approx:.3f} Å")
else:
    print(f"\nNo singlet-triplet crossing found in scanned range")

print("\n" + "=" * 70)
print("Calculation completed successfully!")
print(f"Output files: {csv_filename}, C2_PES_FCI_Singlet_Triplet.pdf")
print("=" * 70)
